# [AHA! Activity Health Analytics](http://casas.wsu.edu/)
[Center for Advanced Studies of Adaptive Systems (CASAS)](http://casas.wsu.edu/)

[Washington State University](https://wsu.edu)

# L7 Virtual Classifier: Part 2

## Learner Objectives
At the conclusion of this lesson, participants should have an understanding of:
* Finishing implementing steps of PACD with the virtual classifier algorithm
    * Class labeling
    * Decision tree training
    * Change significance testing
* Applying the virtual classifier algorithm to synthetic datasets

## Acknowledgments
Content used in this lesson is based upon information in the following sources:
* [Sprint et al., 2016](http://www.sciencedirect.com/science/article/pii/S1532046416300740)
* [Hido et al., 2008](https://link.springer.com/chapter/10.1007%2F978-3-540-68125-0_15)
* [sci-kit learn](http://scikit-learn.org) machine learning library

## VC Example (Continued)
In the previous lesson, wrote code to do the following:
1. Read [fitbit_example_steps_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_example_steps_df.csv) into a data frame
1. Compute daily features and store them in a data frame
1. Write the features data frame to the file [fitbit_example_features_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_example_features_df.csv)

We are going to pick up where we left off by reading [fitbit_example_features_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_example_features_df.csv) into a data frame.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp

fname = r"files\fitbit_example_data_features_df.csv"
features_df = pd.read_csv(fname, header=0, index_col=[0])

### Label Windows
Now that we have feature vectors (rows in our features data frame) for each day, we are ready to label our feature vectors in each window as positive or negative classes:
1. $W_{1}$: positive class
1. $W_{8}$: negative class

In [2]:
def label_windows(features_df): 
    '''
    features_df is a data frame with string dates as the index and features as the columns
    '''
    features_df["class"] = np.NaN
    features_df.loc[0:7, "class"] = 1
    features_df.loc[7:, "class"] = -1

    W1 = features_df.iloc[0:7]
    W2 = features_df.iloc[7:14]
    return W1, W2

W1, W2 = label_windows(features_df)
print(len(W1) == len(W2))

True


### Train the Binary Classifier
Now that we have each day in our windows labeled as a positive or negative class, it's time to train a binary classifier to discriminate between the windows! To do this, we are going to use the [sci-kit learn](http://scikit-learn.org) machine learning library. Specifically, we will train an instance of `sci-kit learn`'s [`DecisionTreeClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) in K fold cross validation. In sci-kit learn, K fold cross validation is computed with the `cross_val_score()` function, which by default for a `DecisionTreeClassifier` evaluates the model by mean accuracy, which is what we want!.

In [3]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import sklearn.tree as tree
from sklearn.tree import DecisionTreeClassifier

def compute_VC_change_score(W1, W2):
    '''
    W1 and W2 are dataframes with string dates as the index and features as columns
    includes class as a column
    '''
    W_concat = pd.concat((W1, W2))
    W_concat_Y = W_concat["class"]
    W_concat_X = W_concat.drop("class", axis=1)

    y = features_df
    clf = DecisionTreeClassifier(random_state=0)
    cv_scores = cross_val_score(clf, W_concat_X, W_concat_Y, cv=7)
    mean_cv_score = np.mean(cv_scores)
    return mean_cv_score

p_VC = compute_VC_change_score(W1, W2)
print(p_VC)

0.428571428571


### Significance Testing
Significance testing of change score $CS$ is necessary to interpret change score values. For the VC approach, Hido and colleagues proposed a test of significance to determine if $p_{VC}$ is significantly greater than $p_{rand}$. For this test, the inverse survival function of a binomial distribution is used to determine a critical value, $p_{critical}$, at which $n$ Bernoulli trials are expected to exceed $p_{rand}$ at $\alpha$ significance. If $p_{VC} \geq p_{critical}$, a significant change exists between the two windows, $W_{i}$ and $W_{j}$.

In [4]:
def compute_binomial_critical_probability(N1, N2, p_bin, alpha):
    '''
    https://oneau.wordpress.com/2011/02/28/simple-statistics-with-scipy/
    '''
    N = N1 + N2
    critical_value = sp.stats.binom.isf(alpha, N, p_bin)
    critical_probability = critical_value / N
    return critical_probability

# for equal length arrays, binomial maximum likelihood will be 0.5
p_bin = 0.5
p_critical = compute_binomial_critical_probability(len(W1), len(W2), p_bin, 0.05)
is_sig = 1 if p_VC > p_critical else 0
print("p_bin: %.2f p_VC: %.2f p_critical: %.2f is_sig: %d" %(p_bin, p_VC, p_critical, is_sig))

p_bin: 0.50 p_VC: 0.43 p_critical: 0.71 is_sig: 0


The results indicate that the VC obtained an average prediction accuracy of 0.43, which is worse than random! The prediction accuracy needs to be greater than or equal to the critical value (0.71), which is not the case: $0.43 \ngeq 0.71$. We have seen an example where VC does not capture significant changes in the data. Let's take a look at a few examples where VC does capture significant changes.

## Additional Examples
To more holistically see how the VC approach captures pattern changes, let's take a look at synthetic Fitbit data. To generate synthetic Fitbit data, we re-sampled step data collected from a volunteer wearing a Fitbit Charge Heart Rate fitness tracker and modified the data to produce four different hybrid-synthetic (HS) physical activity profiles, each exhibiting a different type of change. The length of HS profiles is set to 14 days, resulting in two equal size windows of 7 days for comparison (days 1-7 compared to days 8-14). The HS profiles with their profile identification (HS1-4) and a description are as follows:
1. HS1: Medium day-to-day change and consequently significant window-to-window change. Increased bout duration and intensity from day-to-day.
1. HS2: No significant day-to-day change but significant window-to-window change. Increased activity for days 7-12.
1. HS3: Medium day-to-day change and consequently significant window-to-window change. Increased activity variability from day-to-day.
1. HS4: No significant day-to-day change for days 1-6. Significant day-to-day activity variability for days 7-12. Consequently significant window-to-window change.

### Hybrid-Synthetic Profile 1
[fitbit_hybrid1_steps_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_hybrid1_steps_df.csv)
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/ADM_hybrid1.png" width="700">

In [5]:
def compute_features_df(df):
    '''
    df is a dataframe with a DateTimeIndex and columns corresponding to dates
    '''
    features_df = pd.DataFrame(index=df.columns)
    features_df["total"] = df.sum()
    features_df["max"] = df.max()
    features_df["mean"] = df.mean()
    features_df["std"] = df.std()

    # PHYSICAL ACTIVITY INTENSITY FEATURES
    intensities = ["sedentary", "low", "moderate", "high"]
    # add blank columns for each intensity. values will be filled in one at a time
    for intensity in intensities:
        features_df[intensity] = np.NaN
    
    # the largest step count in the data set
    max_val = features_df["max"].max()
    # need to adjust if resample data using sum instead of mean
    # exclusive of left, inclusive of right
    bins = [-1, 4, 39, 99, max_val]

    for date in df.columns:
        intensity_ser = pd.cut(df[date], bins, labels=intensities)
        counts = pd.value_counts(intensity_ser)
        for intensity in intensities:
            percentage = counts.loc[intensity] / len(df) * 100
            features_df.ix[date][intensity] = percentage # use loc because counts is a categorial index
    return features_df

def run_virtual_classifier(fname):
    '''
    
    '''
    df = pd.read_csv(fname, header=0, index_col=[0])
    features_df = compute_features_df(df)
    W1, W2 = label_windows(features_df)
    p_VC = compute_VC_change_score(W1, W2)
    p_bin = 0.5
    p_critical = compute_binomial_critical_probability(len(W1), len(W2), p_bin, 0.05)
    is_sig = 1 if p_VC > p_critical else 0
    print("p_bin: %.2f p_VC: %.2f p_critical: %.2f is_sig: %d" %(p_bin, p_VC, p_critical, is_sig))
    
fname = r"files\fitbit_hybrid1_data_steps_df.csv"
run_virtual_classifier(fname)

p_bin: 0.50 p_VC: 0.93 p_critical: 0.71 is_sig: 1


### Hybrid-Synthetic Profile 2
[fitbit_hybrid2_steps_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_hybrid2_steps_df.csv)
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/ADM_hybrid2.png" width="700">

In [6]:
fname = r"files\fitbit_hybrid2_data_steps_df.csv"
run_virtual_classifier(fname)

p_bin: 0.50 p_VC: 1.00 p_critical: 0.71 is_sig: 1


### Hybrid-Synthetic Profile 3
[fitbit_hybrid3_steps_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_hybrid3_steps_df.csv)
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/ADM_hybrid3.png" width="700">

In [16]:
fname = r"files\fitbit_hybrid3_data_steps_df.csv"
run_virtual_classifier(fname)

p_bin: 0.50 p_VC: 0.93 p_critical: 0.71 is_sig: 1


### Hybrid-Synthetic Profile 4
[fitbit_hybrid4_steps_df.csv](https://raw.githubusercontent.com/gsprint23/aha/master/lessons/files/fitbit_hybrid4_steps_df.csv)
<img src="https://raw.githubusercontent.com/gsprint23/aha/master/lessons/figures/ADM_hybrid4.png" width="700">

In [17]:
fname = r"files\fitbit_hybrid4_data_steps_df.csv"
run_virtual_classifier(fname)

p_bin: 0.50 p_VC: 1.00 p_critical: 0.71 is_sig: 1
